In [1]:
from dotenv import dotenv_values
from pprint import pprint

from espn_api.football import League

from fantasy_sim.matchup import LeagueWeek


In [2]:
config = dotenv_values('.env')

In [3]:
league_id = 76723515
year = 2023

In [4]:
league = League(league_id=league_id, year=year, espn_s2=config['espn_s2'], swid=config['SWID'])

In [5]:
league

League(76723515, 2023)

In [6]:
league.load_roster_week(1)

In [7]:
league.teams

[Team(CeeDee Lambchops),
 Team(Let 'Em Cook),
 Team(Baby Chark doo-doo),
 Team(Gibbs you hell),
 Team(Burrowing to Last Place),
 Team(Out of Favre),
 Team(Mixon Dixons),
 Team(Di**er?! I hardly know 'er!)]

In [8]:
my_team = league.teams[1]

In [9]:
my_team.roster

[Player(Christian McCaffrey),
 Player(Garrett Wilson),
 Player(Patrick Mahomes),
 Player(Najee Harris),
 Player(Amari Cooper),
 Player(DeAndre Hopkins),
 Player(Rachaad White),
 Player(Kyle Pitts),
 Player(Tyler Lockett),
 Player(Dalvin Cook),
 Player(George Pickens),
 Player(Justin Tucker),
 Player(Saints D/ST),
 Player(Jakobi Meyers),
 Player(Raheem Mostert),
 Player(Dalton Schultz),
 Player(Aaron Rodgers),
 Player(Dolphins D/ST),
 Player(Darnell Mooney)]

In [10]:
box_scores = league.box_scores(1)

In [11]:
box_scores[0]

Box Score(Team(Out of Favre) at Team(Burrowing to Last Place))

In [12]:
box_scores

[Box Score(Team(Out of Favre) at Team(Burrowing to Last Place)),
 Box Score(Team(Baby Chark doo-doo) at Team(Gibbs you hell)),
 Box Score(Team(CeeDee Lambchops) at Team(Mixon Dixons)),
 Box Score(Team(Di**er?! I hardly know 'er!) at Team(Let 'Em Cook))]

In [13]:
league_week = LeagueWeek(box_scores)

In [14]:
print(league_week)

Home Team: Team(Burrowing to Last Place): 155.560 pts
    QB:    Daniel Jones - 4.460
    RB1:   Aaron Jones - 26.700
    RB2:   Bijan Robinson - 20.300
    WR1:   Zay Jones - 16.500
    WR2:   A.J. Brown - 14.900
    TE:    George Kittle - 4.900
    FLEX1: David Montgomery - 13.400
    FLEX2: Khalil Herbert - 11.400
    D/ST:  Cowboys D/ST - 35.000
    K:     Harrison Butker - 8.000
----------
Away Team: Team(Out of Favre): 122.440 pts
    QB:    Trevor Lawrence - 17.740
    RB1:   Derrick Henry - 13.900
    RB2:   Miles Sanders - 11.800
    WR1:   Courtland Sutton - 13.200
    WR2:   Davante Adams - 12.600
    TE:    T.J. Hockenson - 11.500
    FLEX1: Deebo Samuel - 11.300
    FLEX2: Cole Kmet - 9.400
    D/ST:  Packers D/ST - 14.000
    K:     Younghoe Koo - 7.000
--------------------
--------------------
Home Team: Team(Gibbs you hell): 137.520 pts
    QB:    Anthony Richardson - 20.920
    RB1:   Tony Pollard - 22.200
    RB2:   Javonte Williams - 9.700
    WR1:   Michael Pittman 